# MPIA Arxiv on Deck 2

Contains the steps to produce the paper extractions.

In [1]:
# Imports
import os
from IPython.display import Markdown, display
from tqdm.notebook import tqdm
import warnings
from PIL import Image 
import re

# requires arxiv_on_deck_2

from arxiv_on_deck_2.arxiv2 import (get_new_papers, 
                                    get_paper_from_identifier,
                                    retrieve_document_source, 
                                    get_markdown_badge)
from arxiv_on_deck_2 import (latex,
                             latex_bib,
                             mpia,
                             highlight_authors_in_list)

# Sometimes images are really big
Image.MAX_IMAGE_PIXELS = 1000000000 

In [2]:
# Some useful definitions.

class AffiliationWarning(UserWarning):
    pass

class AffiliationError(RuntimeError):
    pass

def validation(source: str):
    """Raises error paper during parsing of source file
    
    Allows checks before parsing TeX code.
    
    Raises AffiliationWarning
    """
    check = mpia.affiliation_verifications(source, verbose=True)
    if check is not True:
        raise AffiliationError("mpia.affiliation_verifications: " + check)

        
warnings.simplefilter('always', AffiliationWarning)


def get_markdown_qrcode(paper_id: str):
    """ Generate a qrcode to the arxiv page using qrserver.com
    
    :param paper: Arxiv paper
    :returns: markdown text
    """
    url = r"https://api.qrserver.com/v1/create-qr-code/?size=100x100&data="
    txt = f"""<img src={url}"https://arxiv.org/abs/{paper_id}">"""
    txt = '<div id="qrcode">' + txt + '</div>'
    return txt


def clean_non_western_encoded_characters_commands(text: str) -> str:
    """ Remove non-western encoded characters from a string
    List may need to grow.
    
    :param text: the text to clean
    :return: the cleaned text
    """
    text = re.sub(r"(\\begin{CJK}{UTF8}{gbsn})(.*?)(\\end{CJK})", r"\2", text)
    return text


def get_initials(name: str) -> str:
    """ Get the short name, e.g., A.-B. FamName
    :param name: full name
    :returns: initials
    """
    initials = []
    # account for non western names often in ()
    if '(' in name:
        name = clean_non_western_encoded_characters_commands(name)
        suffix = re.findall(r"\((.*?)\)", name)[0]
        name = name.replace(f"({suffix})", '')
    else:
        suffix = ''
    split = name.split()
    for token in split[:-1]:
        if '-' in token:
            current = '-'.join([k[0] + '.' for k in token.split('-')])
        else:
            current = token[0] + '.'
        initials.append(current)
    initials.append(split[-1].strip())
    if suffix:
        initials.append(f"({suffix})")
    return ' '.join(initials)

## get list of arxiv paper candidates

We use the MPIA mitarbeiter list webpage from mpia.de to get author names
We then get all new papers from Arxiv and match authors

In [3]:
# deal with the author list and edge cases of people that cannot be consistent on their name  

def filter_non_scientists(name: str) -> bool:
    """ Loose filter on expected authorships

    removing IT, administration, technical staff
    :param name: name
    :returns: False if name is not a scientist
    """
    remove_list = ['Licht', 'Binroth', 'Witzel', 'Jordan',
                   'Zähringer', 'Scheerer', 'Hoffmann', 'Düe',
                   'Hellmich', 'Enkler-Scharpegge', 'Witte-Nguy',
                   'Dehen', 'Beckmann', 'Jager', 'Jäger'
                  ]

    for k in remove_list:
        if k in name:
            return False
    return True

def add_author_to_list(author_list: list) -> list:
    """ Add author to list if not already in list
    
    :param author: author name
    :param author_list: list of authors
    :returns: updated list of authors
    """
    add_list = ['T. Henning']

    for author in add_list:
        if author not in author_list:
            author_list.append(author)
    return author_list

# get list from MPIA website
# filter for non-scientists (mpia.get_mpia_mitarbeiter_list() does some filtering)
mpia_authors = [k[1] for k in mpia.get_mpia_mitarbeiter_list() if filter_non_scientists(k[1])]
# add some missing author because of inconsistencies in their MPIA name and author name on papers
mpia_authors = add_author_to_list(mpia_authors)

In [4]:
new_papers = get_new_papers()
# add manual references
add_paper_refs = []
new_papers.extend([get_paper_from_identifier(k) for k in add_paper_refs])

def robust_call(fn, value, *args, **kwargs):
    try:
        return fn(value, *args, **kwargs)
    except Exception:
        return value

candidates = []
for paperk in new_papers:
    # Check author list with their initials
    normed_author_list = [robust_call(mpia.get_initials, k) for k in paperk['authors']]
    hl_authors = highlight_authors_in_list(normed_author_list, mpia_authors, verbose=True)
    matches = [(hl, orig) for hl, orig in zip(hl_authors, paperk['authors']) if 'mark' in hl]
    paperk['authors'] = hl_authors
    if matches:
        # only select paper if an author matched our list
        candidates.append(paperk)
print("""Arxiv has {0:,d} new papers today""".format(len(new_papers)))        
print("""          {0:,d} with possible author matches""".format(len(candidates)))

M. Zhang  ->  M. Zhang  |  ['M. Zhang']
M. S. d. Joode  ->  M. S. d. Joode  |  ['M. S. d. Joode']
N. Wang  ->  N. Wang  |  ['N. Wang']
S. Jiao  ->  S. Jiao  |  ['S. Jiao']
Arxiv has 48 new papers today
          4 with possible author matches


# Parse sources and generate relevant outputs

From the candidates, we do the following steps:
* get their tarball from ArXiv (and extract data)
* find the main .tex file: find one with \documentclass{...} (sometimes it's non trivial)
* Check affiliations with :func:`validation`, which uses :func:`mpia.affiliation_verifications`
* If passing the affiliations: we parse the .tex source
   * inject sub-documents into the main (flatten the main document)
   * parse structure, extract information (title, abstract, authors, figures...)
   * handles `\graphicspath` if provided
* Generate the .md document.

In [5]:
documents = []
failed = []
for paper in tqdm(candidates):
    # debug crap
    paper['identifier'] = paper['identifier'].lower().replace('arxiv:', '').replace(r'\n', '').strip()
    paper_id = paper['identifier']
    
    folder = f'tmp_{paper_id}'

    try:
        if not os.path.isdir(folder):
            folder = retrieve_document_source(f"{paper_id}", f'tmp_{paper_id}')
        
        try:
            doc = latex.LatexDocument(folder, validation=validation)    
        except AffiliationError as affilerror:
            msg = f"ArXiv:{paper_id:s} is not an MPIA paper... " + str(affilerror)
            failed.append((paper, "affiliation error: " + str(affilerror) ))
            continue
        
        # Hack because sometimes author parsing does not work well
        if (len(doc.authors) != len(paper['authors'])):
            doc._authors = paper['authors']
        else:
            # highlight authors (FIXME: doc.highlight_authors)
            # done on arxiv paper already
            doc._authors = highlight_authors_in_list(
                [get_initials(k) for k in doc.authors], 
                mpia_authors, verbose=True)
        if (doc.abstract) in (None, ''):
            doc._abstract = paper['abstract']
            
        doc.comment = (get_markdown_badge(paper_id) + 
                       "<mark>Appeared on: " + paper['date'] + "</mark> - ")
        if paper['comments']:
            doc.comment += " _" + paper['comments'] + "_"
        
        full_md = doc.generate_markdown_text()
        
        full_md += get_markdown_qrcode(paper_id)
        
        # replace citations
        try:
            bibdata = latex_bib.LatexBib.from_doc(doc)
            full_md = latex_bib.replace_citations(full_md, bibdata)
        except Exception as e:
            print("Issues with the citations")
            print(e)
        
        documents.append((paper_id, full_md))
    except Exception as e:
        warnings.warn(latex.LatexWarning(f"{paper_id:s} did not run properly\n" +
                                         str(e)
                                        ))
        failed.append((paper, "latex error " + str(e)))

  0%|          | 0/4 [00:00<?, ?it/s]

Retrieving document from  https://arxiv.org/e-print/2507.18002


extracting tarball to tmp_2507.18002...

 done.
Retrieving document from  https://arxiv.org/e-print/2507.18185
extracting tarball to tmp_2507.18185...

 done.
Retrieving document from  https://arxiv.org/e-print/2507.18187


extracting tarball to tmp_2507.18187... done.
Retrieving document from  https://arxiv.org/e-print/2507.18547
extracting tarball to tmp_2507.18547...

 done.


S. Jiao  ->  S. Jiao  |  ['S. Jiao']


Issues with the citations
list index out of range


### Export the logs

Throughout, we also keep track of the logs per paper. see `logs-{today date}.md` 

In [6]:
import datetime
today = str(datetime.date.today())
logfile = f"_build/html/logs/log-{today}.md"


with open(logfile, 'w') as logs:
    # Success
    logs.write(f'# Arxiv on Deck 2: Logs - {today}\n\n')
    logs.write("""* Arxiv had {0:,d} new papers\n""".format(len(new_papers)))
    logs.write("""    * {0:,d} with possible author matches\n\n""".format(len(candidates)))
    logs.write("## Sucessful papers\n\n")
    display(Markdown("## Successful papers"))
    success = [k[0] for k in documents]
    for candid in candidates:
        if candid['identifier'].split(':')[-1] in success:
            display(candid)
            logs.write(candid.generate_markdown_text() + '\n\n')

    ## failed
    logs.write("## Failed papers\n\n")
    display(Markdown("## Failed papers"))
    failed = sorted(failed, key=lambda x: x[1])
    current_reason = ""
    for paper, reason in failed:
        if 'affiliation' in reason:
            color = 'green'
        else:
            color = 'red'
        data = Markdown(
                paper.generate_markdown_text() + 
                f'\n|<p style="color:{color:s}"> **ERROR** </p>| <p style="color:{color:s}">{reason:s}</p> |'
               )
        if reason != current_reason:
            logs.write(f'### {reason:s} \n\n')
            current_reason = reason
        logs.write(data.data + '\n\n')
        
        # only display here the important errors (all in logs)
        # if color in ('red',):
        display(data)

## Successful papers


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-2507.18547-b31b1b.svg)](https://arxiv.org/abs/2507.18547) | **An S-shaped filament formed due to Cloud-Cloud Collision in molecular cloud G178.28-00.61**  |
|| T. Zhang, et al. -- incl., <mark>S. Jiao</mark> |
|*Appeared on*| *2025-07-25*|
|*Comments*| **|
|**Abstract**|            We present compelling observational evidence supporting G178.28-00.61 as an early-stage candidate for Cloud-Cloud Collision (CCC), with indications of the formation of an S-shaped filament, evenly-separated dense cores, and young star clusters. The observations of CO molecular line emission demonstrate the existence of two interacting molecular clouds with systemic velocities of 0.8 km/s and -1.2 km/s, respectively. The convergence zone of these two clouds reveals an S-shaped filament in the JCMT 850 micron continuum image, suggesting cloud interaction. In line with expectations from CCC simulations, broad bridging features are discernible in the position-velocity diagrams. An elevated concentration of identified Class I and II young stellar objects along the filament at the intersection area further supports the hypothesis of a collision-induced origin. This observation could be explained by a recent MHD model of CCC (Kong et al. 2024), which predicts a similar morphology, scale, density, and unbound status, as well as the orientation of the polarization.         |

## Failed papers


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-2507.18185-b31b1b.svg)](https://arxiv.org/abs/2507.18185) | **Multi-scale radiative-transfer model of the protoplanetary disc DoAr 44**  |
|| <mark>M. S. d. Joode</mark>, M. Brož |
|*Appeared on*| *2025-07-25*|
|*Comments*| *13 pages, 11 figures. Accepted by Astronomy & Astrophysics*|
|**Abstract**|            We aim to construct a comprehensive global multi-scale kinematic equilibrium radiative-transfer model for the pre-transitional disc of DoAr 44 (Haro 1-16, V2062 Oph) in the Ophiuchus star-forming region. This model integrates diverse observational datasets to describe the system, spanning from the accretion region to the outer disc. Our analysis utilises a large set of observational data, including ALMA continuum complex visibilities, VLTI/GRAVITY continuum squared visibilities, closure phases, and triple products, as well as VLT/UVES and VLT/X-shooter H-alpha spectra. Additionally, we incorporated absolute flux measurements from ground-based optical observatories, Spitzer, IRAS, the Submillimeter Array, the IRAM, or the ATCA radio telescopes. These data sets were used to constrain the structure and kinematics of the object through radiative-transfer modelling. Our model reveals that the spectral line profiles are best explained by an optically thin spherical inflow/outflow within the co-rotation radius of the star, exhibiting velocities exceeding 380 km/s. The VLTI near-infrared interferometric observations are consistent with an inner disc extending from 0.1 to 0.2 au. The ALMA sub-millimetre observations indicate a dust ring located between 36 and 56 au, probably related to the CO2 condensation line. The global density and temperature profiles derived from our model provide insight into an intermediate disc, located in the terrestrial planet-forming zone, which has not yet been spatially resolved.         |
|<p style="color:green"> **ERROR** </p>| <p style="color:green">affiliation error: mpia.affiliation_verifications: '69117' keyword not found.</p> |


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-2507.18002-b31b1b.svg)](https://arxiv.org/abs/2507.18002) | **CU-JADE: A Method for Traversing Extinction Jumps along the Line-of-Sight**  |
|| S. Zhang, et al. -- incl., <mark>M. Zhang</mark> |
|*Appeared on*| *2025-07-25*|
|*Comments*| *31 pages, 15 figures, 2 tables; accepted for publication in AJ*|
|**Abstract**|            Although interstellar dust extinction serves as a powerful distance estimator, the solar system's location within the Galactic plane complicates distance determinations, especially for molecular clouds (MCs) at varying distances along the line of sight (LoS). The presence of complex extinction patterns along LoS introduces degeneracies, resulting in less accurate distance measurements to overlapping MCs in crowded regions of the Galactic plane. In this study, we develop the CUSUM-based Jump-point Analysis for Distance Estimation (CU-JADE), a novel method designed to help mitigate these observational challenges. The key strengths of CU-JADE include: (1) sensitivity to detect abrupt jumps in $Distance$-$A_{\lambda}$ ($D$-$A$) datasets, (2) minimal systematic errors as demonstrated on both mock and observed data, and (3) the ability to combine CUSUM analysis with multi-wavelength data to improve the completeness of distance measurements for nearby gas structures, even for extinction values as low as $\Delta A_{V} \gtrsim 0.15$ mag. By combining CO survey data with a large sample of stars characterized by high-precision parallaxes and extinctions, we uncovered the multi-layered molecular gas distribution in the high-latitude Cepheus region. We also determined accurate distances to MCs beyond the Cygnus Rift by analyzing the intricate structure of gas and extinction within the Galactic plane. Additionally, we constructed a full-sky 3D extinction map extending to 4 kpc, which provides critical insights into dense interstellar medium components dominated by molecular hydrogen. These results advance our understanding of the spatial distribution and physical properties of MCs across the Milky Way.         |
|<p style="color:green"> **ERROR** </p>| <p style="color:green">affiliation error: mpia.affiliation_verifications: 'Heidelberg' keyword not found.</p> |


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-2507.18187-b31b1b.svg)](https://arxiv.org/abs/2507.18187) | **Results of 15-Year Pulsar Timing of PSR J0007+7303 with Fermi-LAT**  |
|| Z.-x. Yu, et al. -- incl., <mark>N. Wang</mark> |
|*Appeared on*| *2025-07-25*|
|*Comments*| *Accepted for publication in ApJ*|
|**Abstract**|            The study of pulsar glitches provides a unique window into the internal structure and dynamic processes of neutron stars. PSR J0007+7303, a very bright gamma-ray pulsar, is the first pulsar discovered by the Fermi-LAT telescope. In this paper, we present the 15 years of timing results of this pulsar using the Fermi-LAT data. We identified nine glitches, five of which are newly discovered. Among these, two are small glitches, occurring between the three previously reported ones, while the other four are large glitches. The glitches exhibit fractional frequency changes ranging from 15 x 10^-9 to 1238 x 10^-9, with intervals of approximately 1-2 years between events. Uniquely, this pulsar shows no exponential recovery behavior following any glitch, setting it apart from most glitching pulsars. Furthermore, no significant changes were observed in the gamma-ray pulse profile, flux, or phase-averaged spectra before and after glitches, indicating the stability of the pulsar's emission properties despite internal changes. A parametric analysis of the glitches yielded a fractional moment of inertia of the crustal superfluid involved in glitches as 1.06 percent, which matches extremely well with previous statistical work if the non-dissipative entrainment effect is not considered and strongly supports the internal origin of these glitches. These results highlight the distinct glitch behavior of PSR J0007+7303 and offer valuable insights into the crust-superfluid interaction in neutron stars. The physical origin of no exponential recovery is also discussed.         |
|<p style="color:green"> **ERROR** </p>| <p style="color:green">affiliation error: mpia.affiliation_verifications: 'Heidelberg' keyword not found.</p> |

## Export documents

We now write the .md files and export relevant images

In [7]:
def export_markdown_summary(md: str, md_fname:str, directory: str):
    """Export MD document and associated relevant images"""
    import os
    import shutil
    import re

    if (os.path.exists(directory) and not os.path.isdir(directory)):
        raise RuntimeError(f"a non-directory file exists with name {directory:s}")

    if (not os.path.exists(directory)):
        print(f"creating directory {directory:s}")
        os.mkdir(directory)

    fig_fnames = (re.compile(r'\[Fig.*\]\((.*)\)').findall(md) + 
                  re.compile(r'\<img src="([^>\s]*)"[^>]*/>').findall(md))
    print("found figures", fig_fnames)
    for fname in fig_fnames:
        if 'http' in fname:
            # No need to copy online figures
            continue
        if not os.path.exists(fname):
            print("file not found", fname)
            continue
        print("copying ", fname, "to", directory)
        destdir = os.path.join(directory, os.path.dirname(fname))
        destfname = os.path.join(destdir, os.path.basename(fname))
        try:
            os.makedirs(destdir)
        except FileExistsError:
            pass
        shutil.copy(fname, destfname)
    with open(os.path.join(directory, md_fname), 'w') as fout:
        fout.write(md)
    print("exported in ", os.path.join(directory, md_fname))
    [print("    + " + os.path.join(directory,fk)) for fk in fig_fnames]

In [8]:
for paper_id, md in documents:
    export_markdown_summary(md, f"{paper_id:s}.md", '_build/html/')

found figures ['tmp_2507.18547/./N_T_pol.png', 'tmp_2507.18547/./SED_fit.png', 'tmp_2507.18547/./moment1_2_all.png', 'tmp_2507.18547/./rgb_map.png', 'tmp_2507.18547/./color_diagram.png']
copying  tmp_2507.18547/./N_T_pol.png to _build/html/
copying  tmp_2507.18547/./SED_fit.png to _build/html/
copying  tmp_2507.18547/./moment1_2_all.png to _build/html/
copying  tmp_2507.18547/./rgb_map.png to _build/html/
copying  tmp_2507.18547/./color_diagram.png to _build/html/
exported in  _build/html/2507.18547.md
    + _build/html/tmp_2507.18547/./N_T_pol.png
    + _build/html/tmp_2507.18547/./SED_fit.png
    + _build/html/tmp_2507.18547/./moment1_2_all.png
    + _build/html/tmp_2507.18547/./rgb_map.png
    + _build/html/tmp_2507.18547/./color_diagram.png


## Display the papers

Not necessary but allows for a quick check.

In [9]:
[display(Markdown(k[1])) for k in documents];

<div class="macros" style="visibility:hidden;">
$\newcommand{\ensuremath}{}$
$\newcommand{\xspace}{}$
$\newcommand{\object}[1]{\texttt{#1}}$
$\newcommand{\farcs}{{.}''}$
$\newcommand{\farcm}{{.}'}$
$\newcommand{\arcsec}{''}$
$\newcommand{\arcmin}{'}$
$\newcommand{\ion}[2]{#1#2}$
$\newcommand{\textsc}[1]{\textrm{#1}}$
$\newcommand{\hl}[1]{\textrm{#1}}$
$\newcommand{\footnote}[1]{}$
$\newcommand$
$\newcommand$</div>



<div id="title">

# An S-shaped filament formed due to Cloud-Cloud Collision in molecular cloud G178.28-00.61

</div>
<div id="comments">

[![arXiv](https://img.shields.io/badge/arXiv-2507.18547-b31b1b.svg)](https://arxiv.org/abs/2507.18547)<mark>Appeared on: 2025-07-25</mark> - 

</div>
<div id="authors">

T. Zhang, et al. -- incl., <mark>S. Jiao</mark>

</div>
<div id="abstract">

**Abstract:** We present compelling observational evidence supporting G178.28-00.61 as an early-stage candidate for Cloud-Cloud Collision (CCC), with indications of the formation of an S-shaped filament, evenly-separated dense cores, and young star clusters. The observations of CO molecular line emission demonstrate the existence of two interacting molecular clouds with systemic velocities of 0.8 km s $^{-1}$ and -1.2 km s $^{-1}$ , respectively. The convergence zone of these two clouds reveals an S-shaped filament in the JCMT 850 $\mu$ m continuum image, suggesting cloud interaction. In line with expectations from CCC simulations, broad bridging features are discernible in the position-velocity diagrams. An elevated concentration of identified Class I and II young stellar objects along the filament at the intersection area further supports the hypothesis of a collision-induced origin.This observation could be explained by a recent MHD model of CCC (Kong et al. 2024), which predicts a similar morphology, scale, density, and unbound status, as well as the orientation of the polarization.

</div>

<div id="div_fig1">

<img src="tmp_2507.18547/./N_T_pol.png" alt="Fig1.1" width="50%"/><img src="tmp_2507.18547/./SED_fit.png" alt="Fig1.2" width="50%"/>

**Figure 1. -** (a) The 850 $\mu$m continuum emission map of G178.2-0.6 in color scale, detected by the JCMT and Planck. Ten emission peaks are marked, which were subject to subsequent spectral observation using the KVN in single point mode. The cores identified with the dendrogram are delineated by black contours. The JCMT beam size and the corresponding physical scale are shown in the bottom left corner. The dust polarization is shown by the dark blue solid line at the center, roughly perpendicular to the filament. (b) to (d) The pixel-by-pixel SED fitting results for two-component modified grey body spectra, based on continuum images across four bands (70, 160, 250 $\mu$m Herschel data and 850 $\mu$m JCMT+Planck data). (b) The total column density map, overlaid with 850 $\mu$m continuum contours which range over 0.47--0.77 mJy/arcsec$^2$ with a step of 0.1 mJy/arcsec$^2$. (c) and (d): the derived dust temperature of the warm and cold component, respectively. (e) and (f) The example SEDs of core A2-4 with single-component and two-component fit. (*fig:continuum-SED*)

</div>
<div id="div_fig2">

<img src="tmp_2507.18547/./moment1_2_all.png" alt="Fig10" width="100%"/>

**Figure 10. -** Moment 1 maps of $^{13}$CO (1-0) (left panel) and $^{12}$CO (2-1) (right panel) in color scale, with corresponding moment 2 maps overlaid as green contours. The contours for the line widths of $^{13}$CO (1-0) are 1.7--2.5 km s$^{-1}$ with a step of 0.2 km s$^{-1}$, and for $^{12}$CO (2-1) are 3.4--4.0 km s$^{-1}$ with a step of 0.2 km s$^{-1}$. The continuum peaks are labeled and found to locate at the intermediate velocity region. The beam sizes are shown as white circles. (*fig:moments*)

</div>
<div id="div_fig3">

<img src="tmp_2507.18547/./rgb_map.png" alt="Fig2.1" width="50%"/><img src="tmp_2507.18547/./color_diagram.png" alt="Fig2.2" width="50%"/>

**Figure 2. -** Left panel: WISE three-color composite image (RGB: 22 $\mu$m, 12 $\mu$m, 3.4 $\mu$m), overlaid with associated YSOs and 850 $\mu$m continuum contours. Class I YSOs are depicted in yellow diamonds, Class II YSOs in squares, shock knots in circles. The black cross shows the IRAS point source. The continuum peaks are marked as white crosses and names on the side. Right panel: The color–color diagram for the bands 3.4 $\mu$m, 4.6 $\mu$m, and 12 $\mu$m used to classify YSOs associated with G178. The dashed lines denotes the criteria used by \cite{koe12,koe14} to delineate the YSO classes. (*fig:WISE_YSO*)

</div><div id="qrcode"><img src=https://api.qrserver.com/v1/create-qr-code/?size=100x100&data="https://arxiv.org/abs/2507.18547"></div>

# Create HTML index

In [10]:
from datetime import datetime, timedelta, timezone
from glob import glob
import os

files = glob('_build/html/*.md')
days = 7
now = datetime.today()
res = []
for fk in files:
    stat_result = os.stat(fk).st_ctime
    modified = datetime.fromtimestamp(stat_result, tz=timezone.utc).replace(tzinfo=None)
    delta = now.today() - modified
    if delta <= timedelta(days=days):
        res.append((delta.seconds, fk))
res = [k[1] for k in reversed(sorted(res, key=lambda x:x[1]))]
npub = len(res)
print(len(res), f" publications files modified in the last {days:d} days.")
# [ print('\t', k) for k in res ];

149  publications files modified in the last 7 days.


In [11]:
import datetime
from glob import glob

def get_last_n_days(lst, days=1):
    """ Get the documents from the last n days """
    sorted_lst = sorted(lst, key=lambda x: x[1], reverse=True)
    for fname, date in sorted_lst:
        if date >= str(datetime.date.today() - datetime.timedelta(days=days)):
            yield fname

def extract_appearance_dates(lst_file):
    dates = []

    def get_date(line):
        return line\
            .split('Appeared on:')[-1]\
            .split('</mark>')[0].strip()

    for fname in lst:
        with open(fname, 'r') as f:
            found_date = False
            for line in f:
                if not found_date:
                    if "Appeared on" in line:
                        found_date = True
                        dates.append((fname, get_date(line)))
                else:
                    break
    return dates

from glob import glob
lst = glob('_build/html/*md')
days = 7
dates = extract_appearance_dates(lst)
res = list(get_last_n_days(dates, days))
npub = len(res)
print(len(res), f" publications in the last {days:d} days.")

12  publications in the last 7 days.


In [12]:
def create_carousel(npub=4):
    """ Generate the HTML code for a carousel with `npub` slides """
    carousel = ["""  <div class="carousel" """,
                """       data-flickity='{ "autoPlay": 10000, "adaptiveHeight": true, "resize": true, "wrapAround": true, "pauseAutoPlayOnHover": true, "groupCells": 1 }' id="asyncTypeset">"""
                ]
    
    item_str = """    <div class="carousel-cell"> <div id="slide{k}" class="md_view">Content {k}</div> </div>"""
    for k in range(1, npub + 1):
        carousel.append(item_str.format(k=k))
    carousel.append("  </div>")
    return '\n'.join(carousel)

def create_grid(npub=4):
    """ Generate the HTML code for a flat grid with `npub` slides """
    grid = ["""  <div class="grid"> """,
                ]
    
    item_str = """    <div class="grid-item"> <div id="slide{k}" class="md_view">Content {k}</div> </div>"""
    for k in range(1, npub + 1):
        grid.append(item_str.format(k=k))
    grid.append("  </div>")
    return '\n'.join(grid)

In [13]:
carousel = create_carousel(npub)
docs = ', '.join(['"{0:s}"'.format(k.split('/')[-1]) for k in res])
slides = ', '.join([f'"slide{k}"' for k in range(1, npub + 1)])

with open("daily_template.html", "r") as tpl:
    page = tpl.read()
    page = page.replace("{%-- carousel:s --%}", carousel)\
               .replace("{%-- suptitle:s --%}",  "7-day archives" )\
               .replace("{%-- docs:s --%}", docs)\
               .replace("{%-- slides:s --%}", slides)
    
with open("_build/html/index_7days.html", 'w') as fout:
    fout.write(page)

In [14]:
# redo for today
days = 1
res = list(get_last_n_days(dates, days))
npub = len(res)
print(len(res), f" publications in the last day.")

carousel = create_carousel(npub)
docs = ', '.join(['"{0:s}"'.format(k.split('/')[-1]) for k in res])
slides = ', '.join([f'"slide{k}"' for k in range(1, npub + 1)])

with open("daily_template.html", "r") as tpl:
    page = tpl.read()
    page = page.replace("{%-- carousel:s --%}", carousel)\
               .replace("{%-- suptitle:s --%}",  "Daily" )\
               .replace("{%-- docs:s --%}", docs)\
               .replace("{%-- slides:s --%}", slides)
    
# print(carousel, docs, slides)
# print(page)
with open("_build/html/index_daily.html", 'w') as fout:
    fout.write(page)

2  publications in the last day.


In [15]:
# Create the flat grid of the last N papers (fixed number regardless of dates)
from itertools import islice 

npub = 6
res = [k[0] for k in (islice(reversed(sorted(dates, key=lambda x: x[1])), 6))]
print(len(res), f" {npub} publications selected.")

grid = create_grid(npub)
docs = ', '.join(['"{0:s}"'.format(k.split('/')[-1]) for k in res])
slides = ', '.join([f'"slide{k}"' for k in range(1, npub + 1)])

with open("grid_template.html", "r") as tpl:
    page = tpl.read()
    page = page.replace("{%-- grid-content:s --%}", grid)\
               .replace("{%-- suptitle:s --%}",  f"Last {npub:,d} papers" )\
               .replace("{%-- docs:s --%}", docs)\
               .replace("{%-- slides:s --%}", slides)
    
# print(grid, docs, slides)
# print(page)
with open("_build/html/index_npub_grid.html", 'w') as fout:
    fout.write(page)

6  6 publications selected.
